In [1]:
import re
import json
# from google_trans_new import google_translator  
from deep_translator import GoogleTranslator as google_translator

translator = google_translator(source='auto', target='ru')  

'''

'''


class Lines:
    name = ''
    nameMask = ''
    lines = []
    lines2 = [] # копия входных данных
    objLines = []
    text = ""

    def __init__(self, name):
        self.name = name
        self.nextName(self.name)

    def makeObj(self):
        obj1 = {"time":[], "en": '', "ru": ''}
        obj = 0

        for line in self.lines:

            line = line.strip()

            match = re.search(r'(^(\d\d):(\d\d)$)', line)
            if match and len(match.groups()) > 0:

                obj = obj1.copy()

                l = obj['time'].copy()
                l.append(int(match[2])*60+int(match[3]))
                obj['time'] = l

            else:
                line = line.replace('…', '')
                obj['en'] = line

                self.objLines.append(obj)
                pass

        prev = self.objLines[0]
        for i in range(1, len(self.objLines)):
            l = prev['time'].copy()
            l.append(self.objLines[i]['time'][0])
            prev['time'] = l

            prev = self.objLines[i]


        self.objLines = self.objLines[:-1]    

        pass


    def translateList(self):
        l = []
        for e in self.objLines:
            l.append(e['en'])

        max = 3600
        lall = []
        sum = 0
        l1 = []
        for i in range(len(l)):
            e = l[i]
            sum += len(e)
            l1.append(e)
            if sum > max or i == len(l)-1:
                lall.append(l1)
                sum = 0
                l1 = []
        ll = []
        for e in lall:
            self.translateSmall(e, ll)

        # print(ll)
        pass

        k = 0
        for e in self.objLines:
            e['ru'] = ll[k][1]
            k += 1
        self.text = json.dumps([ob for ob in self.objLines], indent=2, ensure_ascii=False)


    def translateSmall(self, l , ll):

        l1 = list(map(lambda x: x.replace('.', '') + '.', l))
        text = '\n\n'.join(l1)
        # print(text)
        # textRu = translator.translate(text, lang_tgt='ru')
        textRu = translator.translate(text)
        # lru = textRu.split('.')
        lru = re.split(r'[\.\?\!]\n\n', textRu)
        count = min(len(l), len(lru))
        for i in range(count):
            ll.append([self.addBreak(l[i]), lru[i].strip()])



            




        pass




    def nextName(self, name):
        '''
        c2022t.txt => c2022json.txt
        '''
        name = self.name
        name2 = 'c1022json.txt'
        match = re.search(r'(^\w\d\d\d\d)[a-zA-Z]+', name)
        if match and len(match.groups()) > 0:
            name2 = match.groups()[0]
        self.nameMask = name2
        print(self.nameMask)

    def read(self):
        with open(self.name, 'r', encoding='utf8') as file:
           for line in file:
                 line2 = line.strip()
                 line2 = re.sub("[.,?!]", '', line2)
                 len(line2) == 0 or self.lines.append(line2)

    def write(self, name, text):
        with open(name, 'w', encoding='utf8') as file:
            file.write(text)

    def addBreak(self, text):
        '''
        doesn't => does n't
        '''
        # text1 = re.sub("n't",  " n't", text)
        text2 = re.sub("[.,?!]", '', text)
        return text2
    
    def replaceBreak(self):
        text = '\n'.join(self.lines)
        pattern = r'\r?\n\d\d\:\d\d\r?\n([a-z])'
        text2 = re.sub(pattern, r' \1', text)
        self.lines = text2.split('\n')



lines = Lines('b2122time.txt')
lines.read()

lines.lines2 = lines.lines.copy()
lines.replaceBreak()
lines.write(lines.nameMask + 'a.txt', '\n'.join(lines.lines))

lines.makeObj()
lines.translateList()

lines.write(lines.nameMask + 'json.txt', lines.text)

print(len(lines.lines), len(lines.lines2), \
'\n',lines.text[-400:])





b2122


### Фильтрация титров

In [8]:
import re

name = "b2124srt.txt"
name2 = "b2124a.txt"
txt1 = ''
txt2 = ''

with open(name, 'r', encoding='utf8') as file:
  txt1 = '\n'.join(file)

def replace(a):
  a = a[0]
  txt = '33'
  pat1 = r'\d\d\:(\d\d):\d\d\,\d\d\d'
  x1 = re.sub(pat1, r'\1', a)
  pat2 = r'\d\d\:\d\d:(\d\d)\,(\d\d\d)'
  x2 = re.sub(pat2, r'\1.\2', a)

  txt = str(round(int(x1)*60+float(x2), 2))
  # txt = txt.replace('.', '\.')
  # print(txt)
  # txt = '333'
  return txt

replace('00:06:46,200')
pattern = r'\d\d\:\d\d:\d\d\,\d\d\d'
txt2 = re.sub(pattern, replace, txt1)
txt3 = txt2
pattern = '\r?\n\d+\r?\n'
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
pattern = r'<i>♪ '
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
pattern = r'♪</i>'
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
pattern = r'align:.+'
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
txt2 = txt3.replace(' --> ', ",\n")

# pattern = r'(\d\.\d+) --> \d\.\d+'
# txt2 = re.sub(pattern, r'\1 ', txt3)

txt3 = txt2
pattern = r'♪</i>'
txt2 = re.sub(pattern, r'', txt3)
with open(name2, 'w', encoding='utf8') as file:
  file.write(txt2)

print(txt2[:100])






1.13,
5.23

("Let It Snow! Let It

Snow! Let it Snow!")



7.33,
10.53 

♪ Oh the weather

outsi


### 2 

In [18]:
import re

name = "b2124srt.txt"
name2 = "b2124a.txt"
txt1 = ''
txt2 = ''

with open(name, 'r', encoding='utf8') as file:
  txt1 = '\n'.join(file)

def replace(a):
  a = a[0]
  txt = '33'
  pat1 = r'\d\d\:(\d\d):\d\d\,\d\d\d'
  x1 = re.sub(pat1, r'\1', a)
  pat2 = r'\d\d\:\d\d:(\d\d)\,(\d\d\d)'
  x2 = re.sub(pat2, r'\1.\2', a)

  txt = str(round(int(x1)*60+float(x2), 2))
  # txt = txt.replace('.', '\.')
  # print(txt)
  # txt = '333'
  return txt

replace('00:06:46,200')
pattern = r'\d\d\:\d\d:\d\d\,\d\d\d'
txt2 = re.sub(pattern, replace, txt1)
txt3 = txt2
pattern = '\r?\n\d+\r?\n'
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
pattern = r'<i>♪ '
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
pattern = r'♪</i>'
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
pattern = r'align:.+'
txt2 = re.sub(pattern, r'', txt3)
txt3 = txt2
# txt2 = txt3.replace(' --> ', ",\n")

pattern = r'(\d\.\d+) --> \d+\.\d+'
txt2 = re.sub(pattern, r'\1 ', txt3)

txt3 = txt2
pattern = r'♪'
txt2 = re.sub(pattern, r'', txt3)
# del spaces and empty strings
txt3 = txt2
l = txt2.split('\n')
l2 = []
for e in l:
  e1 = e.strip()
  if len(e1):
    l2.append(e1)
txt2 = '\n'.join(l2)

# del spaces and empty strings
txt3 = txt2
l = txt2.split('\n')
l2 = []
txtPrev = ''
for e in l:
  pattern = r'^[A-Z\d]'
  match = re.search(pattern, e)
  if match:
    l2.append(e)
  else:
    l2[-1] += ' ' + e  


txt2 = '\n'.join(l2)

with open(name2, 'w', encoding='utf8') as file:
  file.write(txt2)

print(txt2[:100])

1.13
Snow! Let it Snow!")
7.33
Oh the weather
But the fire is
14.1
And since we've
Let it snow, let 


In [23]:
s = '''
She will get hurt.
06:43
Just because she is a girl
01:46
doesn't mean she has to fear Tofu.
02:49
Girls also have 
03:50
a lot of courage and strength.
04:53
Do you know of a girl like that?
'''

print(s)


She will get hurt.
06:43
Just because she is a girl
01:46
doesn't mean she has to fear Tofu.
02:49
Girls also have 
03:50
a lot of courage and strength.
04:53
Do you know of a girl like that?



In [60]:
import re


pattern = r'\r?\n\d\d\:\d\d\r?\n([a-z])'
s2 = re.sub(pattern, r' \1', s)

print(s2)





She will get hurt.
06:43
Just because she is a girl doesn't mean she has to fear Tofu.
02:49
Girls also have  a lot of courage and strength.
04:53
Do you know of a girl like that?



In [56]:
import re


pattern = r'\r\n\d\d\:\d\d\r\n[a-z]'
pattern = r'\r?\n(\d\d)'
pattern = r'\r?\n(\d\d):\d\d'
for m in re.finditer(pattern, s):
    print(m.start(), m[0], m[1])



19 
06:43 06
52 
01:46 01
93 
02:49 02
116 
03:50 03
153 
04:53 04


In [47]:
match = re.search(pattern, s[26:])
print(match.groups()[0])
match = re.search(pattern, s[65:])
print(match.groups()[0])


01:46
02:49


In [53]:
t = '111221112211122'
for m in re.finditer(r'22', t):
    print(m.start(), m[0])


3 22
8 22
13 22


In [1]:
import re
t = '111221112211122'
t2 = re.sub(r'2(2)', r'3\1', t)
print(t2)

111321113211132
